# Baseline: Pre-optimized Santa 2025 Submission

This notebook establishes the baseline by using the pre-optimized submission from snapshots.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import math
import os

# Tree geometry (15-vertex polygon)
TREE_VERTICES = [
    (0, 0.8),       # tip
    (-0.125, 0.5),  # tier 1 left
    (-0.05, 0.5),   # tier 1 inner left
    (-0.2, 0.25),   # tier 2 left
    (-0.1, 0.25),   # tier 2 inner left
    (-0.35, 0),     # tier 3 left
    (-0.075, 0),    # trunk top left
    (-0.075, -0.2), # trunk bottom left
    (0.075, -0.2),  # trunk bottom right
    (0.075, 0),     # trunk top right
    (0.35, 0),      # tier 3 right
    (0.1, 0.25),    # tier 2 inner right
    (0.2, 0.25),    # tier 2 right
    (0.05, 0.5),    # tier 1 inner right
    (0.125, 0.5),   # tier 1 right
]

def create_tree_polygon(x, y, angle_deg):
    """Create a tree polygon at position (x, y) with rotation angle_deg."""
    poly = Polygon(TREE_VERTICES)
    poly = rotate(poly, angle_deg, origin=(0, 0))
    poly = translate(poly, x, y)
    return poly

def get_bounding_box_side(polygons):
    """Get the side length of the square bounding box containing all polygons."""
    if not polygons:
        return 0
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    return max(width, height)

print("Tree geometry loaded successfully")
print(f"Tree vertices: {len(TREE_VERTICES)} points")

In [ ]:
# Load the pre-optimized submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(baseline_path)
print(f"Loaded {len(df)} rows from baseline submission")
print(df.head(10))

In [ ]:
# Parse the submission format (values prefixed with 's')
def parse_value(val):
    """Parse a value that may be prefixed with 's'."""
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

# Parse all values
df['x_val'] = df['x'].apply(parse_value)
df['y_val'] = df['y'].apply(parse_value)
df['deg_val'] = df['deg'].apply(parse_value)

# Extract N from id (format: NNN_i)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))

print(f"N values range: {df['n'].min()} to {df['n'].max()}")
print(f"\nSample parsed data:")
print(df[['id', 'x_val', 'y_val', 'deg_val', 'n', 'tree_idx']].head(10))

In [ ]:
# Calculate bounding box side length for each N
side_lengths = {}

for n in range(1, 201):
    n_data = df[df['n'] == n]
    if len(n_data) != n:
        print(f"Warning: N={n} has {len(n_data)} trees instead of {n}")
        continue
    
    polygons = []
    for _, row in n_data.iterrows():
        poly = create_tree_polygon(row['x_val'], row['y_val'], row['deg_val'])
        polygons.append(poly)
    
    side = get_bounding_box_side(polygons)
    side_lengths[n] = side

print(f"Calculated side lengths for {len(side_lengths)} N values")

In [ ]:
# Calculate the total score
def calculate_score(side_lengths_dict):
    score = 0
    for n, side in side_lengths_dict.items():
        score += side**2 / n
    return score

total_score = calculate_score(side_lengths)
print(f"\n=== BASELINE SCORE ===")
print(f"Total Score: {total_score:.6f}")
print(f"Target: 68.894234")
print(f"Gap to target: {total_score - 68.894234:.6f}")

# Show per-N contribution
print(f"\n=== Per-N Score Contribution (top 20) ===")
contributions = [(n, side**2/n, side) for n, side in side_lengths.items()]
contributions.sort(key=lambda x: -x[1])
for n, contrib, side in contributions[:20]:
    print(f"N={n:3d}: side={side:.4f}, contribution={contrib:.4f}")

In [ ]:
# Copy baseline to submission folder
import shutil

os.makedirs('/home/submission', exist_ok=True)
shutil.copy(baseline_path, '/home/submission/submission.csv')
print(f"Copied baseline to /home/submission/submission.csv")

# Verify the copy
df_check = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_check)} rows in submission")

In [ ]:
# Save metrics
import json

metrics = {
    'cv_score': total_score,
    'target': 68.894234,
    'gap': total_score - 68.894234,
    'n_values': 200
}

with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Saved metrics to experiments/001_baseline/metrics.json")
print(f"\nFinal baseline score: {total_score:.6f}")